# LIBRARIES

In [1]:
import pandas as pd
import numpy as np
from pkg.constants import *
from pkg.greedy import greedy
from pkg.evaluation import Evaluation

# LECTURA DE DATOS

In [2]:
deltas_df   = pd.read_csv('./00_datos/deltas_5m.csv') 
deltas_matrix   = deltas_df.to_numpy()

# SOLUCIÓN GREEDY

In [3]:
initial_state = deltas_matrix[0,:]
greedy_sol = greedy(initial_state)
greedy_sol.sum()

220.0

In [5]:
greedy_sol

array([ 7.,  9., 18.,  8., 11., 18., 11., 12.,  8., 13., 13., 24., 11.,
       18., 20., 19.])

# FUNCION DE EVALUACION

In [4]:
# 574.5513854177129

evaluation = Evaluation()

evaluation.evaluate(greedy_sol)

574.5513854177129